In [1]:
import numpy as np
import pandas as pd
from skimage.feature import hog
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score
import joblib

from collections import Counter
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
height, width = 64, 64

In [2]:
data = pd.read_csv(f"../archive/ascii_character_classification_{height}_x_{width}.csv", header=0).sample(frac=.05)

label_counts = Counter(data.iloc[:, 0])
print(label_counts)

Counter({0: 12306, 31: 1264, 54: 1254, 49: 1249, 60: 1246, 22: 1245, 56: 1239, 35: 1238, 21: 1238, 55: 1231, 81: 1228, 70: 1224, 71: 1221, 45: 1220, 32: 1219, 89: 1219, 36: 1217, 93: 1217, 79: 1216, 4: 1215, 1: 1215, 91: 1215, 5: 1214, 15: 1211, 37: 1210, 30: 1207, 85: 1207, 86: 1206, 16: 1205, 29: 1205, 51: 1204, 23: 1204, 66: 1204, 44: 1203, 58: 1203, 46: 1201, 57: 1201, 24: 1201, 33: 1199, 87: 1198, 82: 1197, 2: 1195, 64: 1195, 80: 1194, 39: 1193, 26: 1192, 77: 1191, 17: 1190, 42: 1190, 12: 1188, 19: 1187, 8: 1187, 20: 1186, 74: 1185, 38: 1183, 50: 1183, 34: 1182, 72: 1182, 14: 1179, 90: 1178, 83: 1178, 18: 1177, 65: 1177, 62: 1176, 95: 1175, 7: 1173, 10: 1172, 84: 1171, 48: 1171, 78: 1167, 3: 1167, 6: 1165, 76: 1162, 47: 1162, 73: 1162, 61: 1161, 40: 1159, 53: 1158, 92: 1157, 68: 1156, 59: 1155, 41: 1154, 69: 1149, 28: 1148, 9: 1141, 75: 1135, 63: 1134, 13: 1132, 52: 1126, 11: 1125, 43: 1123, 67: 1120, 88: 1118, 27: 1116, 94: 1113, 25: 1089})


In [3]:
X = data.iloc[:, 1:].astype("float64")   # Features are all columns except the first one
y = data.iloc[:, 0].astype("float64")     # Labels are the first column
# Initialize the oversampler


# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
# Optionally, if you want to convert them back to pandas DataFrames:
train_data = pd.concat([y_train, X_train], axis=1)
test_data = pd.concat([y_test, X_test], axis=1)

In [ ]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
joblib.dump(clf, '../artifacts/svm_model_64_x_64.pkl')

In [ ]:
y_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = clf.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

In [ ]:
def extract_hog_features(images):
    hog_features = []
    for image in images:
        image_reshaped = image.reshape((height, width))
        features = hog(image_reshaped, pixels_per_cell=(2, 2), cells_per_block=(1, 1), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

X_hog = extract_hog_features(np.array(X))

X_train, X_test, y_train, y_test = train_test_split(X_hog, y, test_size=0.2, random_state=42)


In [ ]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
joblib.dump(clf, '../artifacts/svm_model_hog_64_x_64.pkl')

In [ ]:
y_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = clf.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")

In [9]:
import cv2

def extract_sift_features(images):
    sift = cv2.SIFT_create()
    sift_features = []
    
    for image in images:
        image_reshaped = image.reshape((height, width)).astype(np.uint8)
        keypoints, descriptors = sift.detectAndCompute(image_reshaped, None)
        
        # If no keypoints are found, use a zero array of the same length as a typical descriptor
        if descriptors is None:
            descriptors = np.zeros((1, sift.descriptorSize()), dtype=np.float32)
        
        # Flatten descriptors and use them as features
        features = descriptors.flatten()
        sift_features.append(features)
    
    return np.array(sift_features)

# Extract SIFT features
X_sift = extract_sift_features(np.array(X))

In [10]:
X_sift = extract_sift_features(np.array(X))

# Since the number of features might vary, we need to ensure consistent feature vector size
# Here, we'll pad with zeros to the maximum descriptor length found
max_len = max(len(f) for f in X_sift)
X_sift = np.array([np.pad(f, (0, max_len - len(f)), 'constant') for f in X_sift])

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_sift, y, test_size=0.2, random_state=42)

In [ ]:
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
joblib.dump(clf, '../artifacts/svm_model_sift_64_x_64.pkl')

In [ ]:
y_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_pred)
y_pred = clf.predict(X_test)
print("test Shape:", X_test.shape)

test_accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f"Train Accuracy: {train_accuracy*100:.4f}%")
print(f"Test Accuracy: {test_accuracy*100:.4f}%")
print(f"F1 Score: {f1*100:.4f}%")
print(f"Recall: {recall*100:.4f}%")